<a href="https://colab.research.google.com/github/Sora-Sugiyama/ML/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install Pillow

In [ ]:
import torch #pytorch 사용
import torch.nn as nn # neural network library from pytorch 사용
import torch.nn.functional as F # pytorch 내부의 F 함수 사용
import torch.optim as optim # optimizer 사용
import matplotlib.pyplot as plt # 시각화를 위한 matplotlib 사용
from torch.utils.data import DataLoader #Data를 batch_size로 나누기 위해 사용
from tqdm.notebook import tqdm
import os
import time
from torch.utils.data import DataLoader, Dataset
import numpy as np
from PIL import Image
import pandas as pd

In [ ]:
# Image Downloader

def ImageLoader(url,name):
  os.system('curl '+url+' > '+'/content/'+name+'.jpeg')
  img=Image.open('/content/'+name+'.jpeg')
  return img

In [ ]:
# drive mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from torchvision.transforms import ToTensor
tf_toTensor = ToTensor()

def label_N_img2tensor(data,st):
  xd,yd=[],[]
  cnt=0
  now=20
  for it in data['0']:
    xd.append(it)

  for it in data['1']:
    cnt+=1
    img=ImageLoader(it,'img'+str(st)) # with out Filename extension
    st+=1
    img_Tensor = tf_toTensor(img.convert('RGB'))
    if(cnt/len(data)*100>=now):
      now+=20
      print(str(round(cnt/len(data)*100,1))+'%..')
      if(cnt==len(data)):print('__Done__')

    yd.append(img_Tensor)
  return xd,yd

In [ ]:
xdata,ydata=[],[]
for i in range(1,6):
  tmpx=[]
  trData=pd.read_csv('/content/drive/MyDrive/data/trainData/data'+str(i)+'.csv')
  tmpx,tmpy=label_N_img2tensor(trData,i*1000)
  mu=sum(tmpx)/len(tmpx)
  sigma=np.std(np.array(tmpx))
  for j in range(len(tmpx)):
    if tmpx[j]<=mu-sigma:tmpx[j]=0
    elif tmpx[j]<mu+sigma:tmpx[j]=1
    else :tmpx[j]=2
  xdata+=tmpx
  ydata+=tmpy

20.6%..
40.5%..
60.3%..
80.2%..
100.0%..
__Done__
20.5%..
40.4%..
60.2%..
80.1%..
100.0%..
__Done__
20.5%..
40.2%..
60.7%..
80.4%..
100.0%..
__Done__
20.0%..
40.0%..
60.0%..
80.0%..
100.0%..
__Done__
20.0%..
40.0%..
60.0%..
80.0%..
100.0%..
__Done__


In [ ]:
class TensorData(Dataset):
    # 외부에 있는 데이터를 가져오기 위해 외부에서 데이터가 들어올 수 있도록, x_data, y_data 변수를 지정
    def __init__(self, x_data, y_data):
        #들어온 x는 tensor형태로 변환
        self.x_data = x_data

        self.y_data = torch.LongTensor(y_data) # float tensor / long tensor 로 숫자 속성을 정해줄 수 있음
        self.len = self.y_data.shape[0]

    # x,y를 튜플형태로 바깥으로 내보내기
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [ ]:
trainData=TensorData(ydata, xdata)

In [ ]:
trainData[0][0].size()

torch.Size([3, 90, 120])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class VGG(nn.Module):
    def __init__(self, config, num_classes=1000, cifar=False):
        super(VGG, self).__init__()
        self.features = make_layer(config)

        # ImageNet
        self.regression = nn.Sequential(
            nn.Linear(3072, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 3)
        )

    def forward(self, x):
        out = self.features(x)
        out = torch.flatten(out,1)
        out = self.regression(out)
        return out

cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64,64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64,64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64,64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

def make_layer(config):
    layers = []
    in_planes = 3
    for value in config:
        if value == "M":
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        else:
            layers.append(nn.Conv2d(in_planes, value, kernel_size=3, padding=1))
            layers.append(nn.ReLU())
            in_planes = value
    return nn.Sequential(*layers)

def VGG11():
    return VGG(config = cfg['A'])

def VGG13():
    return VGG(config = cfg['B'])

def VGG16():
    return VGG(config = cfg['D'])

def VGG19():
    return VGG(config = cfg['E'])

In [ ]:
device = 'cuda'
model = VGG16()
model = model.to(device)
# VGG11, VGG13, VGG16, VGG19 중에 택일하여 사용
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), pad

In [ ]:
epochs = 50
lr = 2e-4

# seed
seed = 2914
torch.manual_seed = seed  #매번 같은 결과 값이 나오게 함

loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
from torchsummary import summary
summary(model, input_size=(3, 90,120))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 64, 90, 120]           1,792
              ReLU-2          [-1, 64, 90, 120]               0
            Conv2d-3          [-1, 64, 90, 120]          36,928
              ReLU-4          [-1, 64, 90, 120]               0
         MaxPool2d-5           [-1, 64, 45, 60]               0
            Conv2d-6          [-1, 128, 45, 60]          73,856
              ReLU-7          [-1, 128, 45, 60]               0
            Conv2d-8          [-1, 128, 45, 60]         147,584
              ReLU-9          [-1, 128, 45, 60]               0
        MaxPool2d-10          [-1, 128, 22, 30]               0
           Conv2d-11          [-1, 256, 22, 30]         295,168
             ReLU-12          [-1, 256, 22, 30]               0
           Conv2d-13          [-1, 256, 22, 30]         590,080
             ReLU-14          [-1, 256,

In [ ]:
train_loader = torch.utils.data.DataLoader(trainData, batch_size=128, shuffle=True)

In [ ]:
teData=pd.read_csv('/content/drive/MyDrive/data/testData/test_data.csv')
teData

,Unnamed: 0,0,1
0,0,1256888,https://i.ytimg.com/vi/dEzMEjXSlgY/default.jpg
1,1,2823725,https://i.ytimg.com/vi/KXkOt0O4VnY/default.jpg
2,2,3221054,https://i.ytimg.com/vi/oq23z2HdCJw/default.jpg
3,3,301372,https://i.ytimg.com/vi/981bWZlkluo/default.jpg
4,4,4175477,https://i.ytimg.com/vi/RBDoN0ehZXE/default.jpg
...,...,...,...
80,80,936939,https://i.ytimg.com/vi/tYumTbttcBU/default.jpg
81,81,204844,https://i.ytimg.com/vi/9Qe7fnRSc0M/default.jpg
82,82,1191582,https://i.ytimg.com/vi/mA4pmFd5h84/default.jpg
83,83,1844467,https://i.ytimg.com/vi/T3lnvBgVCYg/default.jpg


In [ ]:
xdata,ydata=label_N_img2tensor(teData,7000)
mu=sum(xdata)/len(xdata)
sigma=np.std(np.array(xdata))
for j in range(len(xdata)):
  if xdata[j]<=mu-sigma:xdata[j]=0
  elif xdata[j]<mu+sigma:xdata[j]=1
  else :xdata[j]=2

20.0%..
40.0%..
60.0%..
80.0%..
100.0%..
__Done__


In [ ]:
testData=TensorData(ydata, xdata)
testData

In [ ]:
test_loader = torch.utils.data.DataLoader(testData, batch_size=128, shuffle=True)

In [ ]:
tot_train_loss= []

for e in range(1, epochs+1):
    model.train()
    print(f"============= [Epochs] {e}/{epochs} =============")
    train_loss = 0.0
    for datas,labels in tqdm(train_loader):
        datas = datas.to(device)
        labels = labels.to(device)

        output = model(datas)
        loss = loss_fn(output, labels)
        train_loss += loss.item()

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

    tot_train_loss.append(train_loss/len(train_loader))
    print(f"Loss: {train_loss/len(train_loader):.6f}")

    # 매 에폭마다 성능 차이를 보기 위해 실행
    with torch.no_grad(): # autograd 엔진을 비활성화
        model.eval() # dropout을 비활성화 시켜줌
        test_acc = 0.0
        cnt=0
        for datas, labels in tqdm(test_loader):

            datas = datas.to(device)
            labels = labels.to(device)

            output = model(datas)

            preds = torch.argmax(output, dim=1)
            test_acc += preds.eq(labels).sum()

    print(f"[Epoch] {e}, [Test Result] {(test_acc/len(test_loader.dataset))*100:.3f}%")


============= [Epochs] 1/50 =============


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.872355


  0%|          | 0/1 [00:00<?, ?it/s]

[Epoch] 1, [Test Result] 88.235%
============= [Epochs] 2/50 =============


...

[Epoch] 49, [Test Result] 88.235%
============= [Epochs] 50/50 =============


  0%|          | 0/5 [00:00<?, ?it/s]

Loss: 0.349348


  0%|          | 0/1 [00:00<?, ?it/s]

[Epoch] 50, [Test Result] 88.235%


In [ ]:
with torch.no_grad(): # autograd 엔진을 비활성화
    model.eval() # dropout을 비활성화 시켜줌
    test_acc = 0.0
    for datas, labels in tqdm(test_loader):

        datas= datas.to(device)
        labels= labels.to(device)

        output= model(datas)

        preds= torch.argmax(output, dim=1)
        test_acc+= preds.eq(labels).sum()

print(f"[Test Result] {(test_acc/len(test_loader.dataset))*100:.3f}%")

  0%|          | 0/1 [00:00<?, ?it/s]

[Test Result] 88.235%
